In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopy
import nominatim
import requests
from time import sleep
from random import randint
import geopandas as gpd
from shapely.geometry import Point
import datetime

derecho911_df = pd.read_csv('../data/911_Phone Calls_Derecho_050320 050420.csv')
tornado911_df = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
derecho_incidents_df = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv', skiprows=3)
tornado_incidents_df = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows=3)
single_family_housing_df = pd.read_csv('../data/2017 single family_final_with_sqft.txt')

In [79]:
## Filter to get the derech911 dataframe with only ALI NaNs ##
derech_nans = derecho911_df['ALI Latitude'].isna()
derecho911_nans_df = derecho911_df[derech_nans]
tower_not_nans = derecho911_nans_df['Cell Tower Address'].notna()
derecho911_tower_not_lat = derecho911_nans_df[tower_not_nans]
derecho911_full_filter = derecho911_tower_not_lat['Cell Tower Address'] != 'WIRELESS CALLER'
derecho911_fully_filtered = derecho911_tower_not_lat[derecho911_full_filter]

In [80]:
torn_nans = tornado911_df['ALI Latitude'].isna()
tornado911_nans_df = tornado911_df[torn_nans]
torn_tower_not_nans = tornado911_nans_df['Cell Tower Address'].notna()
tornado911_tower_not_lat = tornado911_nans_df[torn_tower_not_nans]

In [81]:
## TEST ANY INDIVIDUAL ADDRESS ##
test_address = '5655 FRIST BLVD'
HOST = 'https://nominatim.openstreetmap.org/search?q=' + test_address + ',Nashville,Tennessee&format=json&polygon=1&addressdetails=1'

In [82]:
test_latlong = '36.1447301,-82.4049953'
HOST1 = 'https://nominatim.openstreetmap.org/search?q=' + test_latlong + '&format=json&polygon=1&addressdetails=1'
req = requests.get(HOST1)
req.text

'[{"place_id":267017387,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":19645526,"boundingbox":["36.144827","36.144927","-82.405045","-82.404945"],"lat":"36.144877","lon":"-82.404995","display_name":"342, Obrien Avenue, Hulan Hollow, Erwin, Unicoi County, Tennessee, 37650, United States of America","class":"place","type":"house","importance":0.001,"address":{"house_number":"342","road":"Obrien Avenue","hamlet":"Hulan Hollow","town":"Erwin","county":"Unicoi County","state":"Tennessee","postcode":"37650","country":"United States of America","country_code":"us"}}]'

In [83]:
def location_retrieve(addresses_in_nash):  ##A list of two-tuples (index, Cell Tower Address)
    locations = {}
    for ind, address in addresses_in_nash:
        piked_address = address.replace('PKE', 'PIKE', 1)
        HOST = 'https://nominatim.openstreetmap.org/search?q='
        HOST = HOST + piked_address
        HOST = HOST + ',Tennessee&format=json&polygon=1&addressdetails=1'
        req = requests.get(HOST)
        req_dict_list = req.json()        
        if len(req_dict_list) > 0:
            ##print(str(req_dict_list))
            result_dict = {}
            result_dict['zip'] = req_dict_list[0]['address']['postcode']
            if 'city' in req_dict_list[0]['address']:                
                result_dict['city'] = req_dict_list[0]['address']['city']
            elif 'village' in req_dict_list[0]['address']:
                result_dict['city'] = req_dict_list[0]['address']['village']
            else:
                result_dict['city'] = req_dict_list[0]['address']['hamlet']
            if 'neighbourhood' in req_dict_list[0]['address']:
                result_dict['hood'] = req_dict_list[0]['address']['neighbourhood']
            else:
                result_dict['hood'] = np.nan
            result_dict['lat'] = req_dict_list[0]['lat']
            result_dict['lon'] = req_dict_list[0]['lon']
            locations[ind] = result_dict
        else:
            locations[ind] = {'zip': 0, 'city': '', 'lat': 0.0, 'lon': 0.0}
        sleep(randint(1,4))
    return locations

#### TEST function using sample data ####
some_addresses = [(5,'5600 OBRIEN AV'), (82, '4220 HARDING PKE')]

locations = location_retrieve(some_addresses)
##print(str(locations))

In [84]:
## Create Test DataFrame ##
data = {'Seizure DateTime':['5/3/2020 18:22', '5/3/2020 18:22'],
        'CallTypeId': ['911 Calls', '911 Calls'], 'ALI Latitude': [np.nan, np.nan], 
        'ALI Longitude': [np.nan, np.nan], 'Cell Tower Address': ['5600 OBRIEN AV', '4220 HARDING PKE'], 'zip': ['37207', 37209]}
test_nan_df = pd.DataFrame(data, index=[5, 82])
##print(test_nan_df.head())

In [85]:
def integrate_latlon(nan_df, df, address_col, lat_col, lon_col):  ## input dataframe and the string values for the column names
    loc = location_retrieve(zip(nan_df.index, nan_df[address_col]))
    for ind in loc:
        df.at[ind, lat_col] = loc[ind]['lat']
        df.at[ind, lon_col] = loc[ind]['lon']
    return df

##TEST##
integrated = integrate_latlon(test_nan_df, test_nan_df, 'Cell Tower Address', 'ALI Latitude', 'ALI Longitude')
##integrated.head()

In [86]:
##Initially used to test before the integrate_latlon function was written
##retrievals = location_retrieve(zip(derecho911_fully_filtered.index, derecho911_fully_filtered['Cell Tower Address']))

derecho911_fill_df = integrate_latlon(derecho911_fully_filtered, derecho911_df, 'Cell Tower Address', 'ALI Latitude', 'ALI Longitude')


In [87]:
tornado911_fill_df = integrate_latlon(tornado911_tower_not_lat, tornado911_df, 'Cell Tower Address', 'ALI Latitude', 'ALI Longitude')

In [88]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

##Make a geometry column which contains points required for spatial join
derecho911_fill_df['geometry'] = derecho911_fill_df.apply(lambda row: Point(row['ALI Longitude'], row['ALI Latitude']), axis=1)
tornado911_fill_df['geometry'] = tornado911_fill_df.apply(lambda row: Point(row['ALI Longitude'], row['ALI Latitude']), axis=1)

##Make a latLong column which contains an array which is required for map markers for folium
derecho911_fill_df['latlong'] = derecho911_fill_df.apply(lambda row: [row['ALI Latitude'], row['ALI Longitude']], axis=1)
tornado911_fill_df['latlong'] = tornado911_fill_df.apply(lambda row: [row['ALI Latitude'], row['ALI Longitude']], axis=1)

tornado_incidents_df['geometry'] = tornado_incidents_df.apply(lambda row: Point(row['Longitude1'], row['Latitude1']), axis=1)
derecho_incidents_df['geometry'] = derecho_incidents_df.apply(lambda row: Point(row['Longitude1'], row['Latitude1']), axis=1)

tornado_incidents_df['latlong'] = tornado_incidents_df.apply(lambda row: [row['Latitude1'], row['Longitude1']], axis=1)
derecho_incidents_df['latlong'] = derecho_incidents_df.apply(lambda row: [row['Latitude1'], row['Longitude1']], axis=1)

t911_geo = gpd.GeoDataFrame(tornado911_fill_df, 
                           crs = zipcodes.crs, 
                           geometry = tornado911_fill_df['geometry'])
t911_gpd = gpd.sjoin(t911_geo, zipcodes, op = 'within')

d911_geo = gpd.GeoDataFrame(derecho911_fill_df, 
                           crs = zipcodes.crs, 
                           geometry = derecho911_fill_df['geometry'])
d911_gpd = gpd.sjoin(d911_geo, zipcodes, op = 'within')

tincidents_geo = gpd.GeoDataFrame(tornado_incidents_df, 
                           crs = zipcodes.crs, 
                           geometry = tornado_incidents_df['geometry'])
tincidents_gpd = gpd.sjoin(tincidents_geo, zipcodes, op = 'within')

dincidents_geo = gpd.GeoDataFrame(derecho_incidents_df, 
                           crs = zipcodes.crs, 
                           geometry = derecho_incidents_df['geometry'])
dincidents_gpd = gpd.sjoin(dincidents_geo, zipcodes, op = 'within')

##simple map example following Mary's intro to geo
##ax = zipcodes.plot(figsize = (8, 10), color = 'lightgreen')
##t911_geo.plot( ax = ax);
##plt.show();

In [89]:
##Convert to DateTime in order to assign a Time_Window
##Windows for Tornado are 12pm to 5am - Windows for Derecho are 4pm to 1230pm next day - each has 5
t911_gpd['DateTime'] = pd.to_datetime(t911_gpd['Seizure DateTime'])

t911_gpd.loc[t911_gpd['DateTime'].between(pd.Timestamp('2020-03-03 00:00:00'), pd.Timestamp('2020-03-03 01:00:00')), 'Time_Window'] = 1
t911_gpd.loc[t911_gpd['DateTime'].between(pd.Timestamp('2020-03-03 01:00:00'), pd.Timestamp('2020-03-03 02:00:00')), 'Time_Window'] = 2
t911_gpd.loc[t911_gpd['DateTime'].between(pd.Timestamp('2020-03-03 02:00:00'), pd.Timestamp('2020-03-03 03:00:00')), 'Time_Window'] = 3
t911_gpd.loc[t911_gpd['DateTime'].between(pd.Timestamp('2020-03-03 03:00:00'), pd.Timestamp('2020-03-03 04:00:00')), 'Time_Window'] = 4
t911_gpd.loc[t911_gpd['DateTime'].between(pd.Timestamp('2020-03-03 04:00:00'), pd.Timestamp('2020-03-03 05:00:00')), 'Time_Window'] = 5

In [90]:
d911_gpd['DateTime'] = pd.to_datetime(d911_gpd['Seizure DateTime'])

d911_gpd.loc[d911_gpd['DateTime'].between(pd.Timestamp('2020-05-03 16:00:00'), pd.Timestamp('2020-05-03 17:30:00')), 'Time_Window'] = 1
d911_gpd.loc[d911_gpd['DateTime'].between(pd.Timestamp('2020-05-03 17:30:00'), pd.Timestamp('2020-05-03 19:00:00')), 'Time_Window'] = 2
d911_gpd.loc[d911_gpd['DateTime'].between(pd.Timestamp('2020-05-03 19:00:00'), pd.Timestamp('2020-05-03 20:30:00')), 'Time_Window'] = 3
d911_gpd.loc[d911_gpd['DateTime'].between(pd.Timestamp('2020-05-03 20:30:00'), pd.Timestamp('2020-05-03 22:00:00')), 'Time_Window'] = 4
d911_gpd.loc[d911_gpd['DateTime'].between(pd.Timestamp('2020-05-03 22:00:00'), pd.Timestamp('2020-05-04 22:00:00')), 'Time_Window'] = 5


In [91]:
tincidents_gpd['DateTime'] = pd.to_datetime(tincidents_gpd['IncidentDate'])

tincidents_gpd.loc[tincidents_gpd['DateTime'].between(pd.Timestamp('2020-03-03 00:00:00'), pd.Timestamp('2020-03-03 01:00:00')), 'Time_Window'] = 1
tincidents_gpd.loc[tincidents_gpd['DateTime'].between(pd.Timestamp('2020-03-03 01:00:00'), pd.Timestamp('2020-03-03 02:00:00')), 'Time_Window'] = 2
tincidents_gpd.loc[tincidents_gpd['DateTime'].between(pd.Timestamp('2020-03-03 02:00:00'), pd.Timestamp('2020-03-03 03:00:00')), 'Time_Window'] = 3
tincidents_gpd.loc[tincidents_gpd['DateTime'].between(pd.Timestamp('2020-03-03 03:00:00'), pd.Timestamp('2020-03-03 04:00:00')), 'Time_Window'] = 4
tincidents_gpd.loc[tincidents_gpd['DateTime'].between(pd.Timestamp('2020-03-03 04:00:00'), pd.Timestamp('2020-03-03 05:00:00')), 'Time_Window'] = 5


In [92]:
dincidents_gpd['DateTime'] = pd.to_datetime(dincidents_gpd['IncidentDate'])

dincidents_gpd.loc[dincidents_gpd['DateTime'].between(pd.Timestamp('2020-05-03 16:00:00'), pd.Timestamp('2020-05-03 17:30:00')), 'Time_Window'] = 1
dincidents_gpd.loc[dincidents_gpd['DateTime'].between(pd.Timestamp('2020-05-03 17:30:00'), pd.Timestamp('2020-05-03 19:00:00')), 'Time_Window'] = 2
dincidents_gpd.loc[dincidents_gpd['DateTime'].between(pd.Timestamp('2020-05-03 19:00:00'), pd.Timestamp('2020-05-03 20:30:00')), 'Time_Window'] = 3
dincidents_gpd.loc[dincidents_gpd['DateTime'].between(pd.Timestamp('2020-05-03 20:30:00'), pd.Timestamp('2020-05-03 22:00:00')), 'Time_Window'] = 4
dincidents_gpd.loc[dincidents_gpd['DateTime'].between(pd.Timestamp('2020-05-03 22:00:00'), pd.Timestamp('2020-05-04 22:00:00')), 'Time_Window'] = 5


In [93]:
derech_map = folium.Map(location=[36.16, -86.72], zoom_start=11)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(derech_map)

for ind, row in d911_gpd.iterrows():
    color='white'
    ##From blue to green::early - late
    if row['Time_Window'] == 1:
        color='#5d4eff'
    elif row['Time_Window'] == 2:
        color='#52a4ff'
    elif row['Time_Window'] == 3:
        color='#00f9ff'
    elif row['Time_Window'] == 4:
        color='#50ff7f'
    elif row['Time_Window'] == 5:
        color='#44ff32'
    folium.Circle(location=row['latlong'], color=color, opacity=0.7, radius=150).add_to(derech_map)
derech_map

In [94]:
torn_map = folium.Map(location=[36.16, -86.72], zoom_start=12)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(torn_map)

for ind, row in t911_gpd.iterrows():
    color='white'
    ##From light red to dark red::early - late
    if row['Time_Window'] == 1:
        color='#ffb3b3'
    elif row['Time_Window'] == 2:
        color='#ff8080'
    elif row['Time_Window'] == 3:
        color='#ff4d4d'
    elif row['Time_Window'] == 4:
        color='#ff1a1a'
    elif row['Time_Window'] == 5:
        color='#e60000'
    folium.Circle(location=row['latlong'], color=color, radius=80).add_to(torn_map)
torn_map

In [95]:
derech_inc_map = folium.Map(location=[36.16, -86.72], zoom_start=11)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(derech_inc_map)

for ind, row in dincidents_gpd.iterrows():
    color='white'
    if row['Time_Window'] == 1:
        color='#5d4eff'
    elif row['Time_Window'] == 2:
        color='#52a4ff'
    elif row['Time_Window'] == 3:
        color='#00f9ff'
    elif row['Time_Window'] == 4:
        color='#50ff7f'
    elif row['Time_Window'] == 5:
        color='#44ff32'
    folium.Circle(location=row['latlong'], color=color, radius=80, tooltip=dincidents_gpd.loc[ind,'IncidentTypeDescription1']).add_to(derech_inc_map)
derech_inc_map

In [96]:
torn_inc_map = folium.Map(location=[36.16, -86.72], zoom_start=11)
##This is an arbitrary location within Nashville to start; this could be more centered
folium.TileLayer('stamentoner').add_to(torn_inc_map)

for ind, row in tincidents_gpd.iterrows():
    color='white'
    if row['Time_Window'] == 1:
        color='#5d4eff'
    elif row['Time_Window'] == 2:
        color='#52a4ff'
    elif row['Time_Window'] == 3:
        color='#00f9ff'
    elif row['Time_Window'] == 4:
        color='#50ff7f'
    elif row['Time_Window'] == 5:
        color='#44ff32'
    folium.Circle(location=row['latlong'], color=color, radius=80, tooltip=tincidents_gpd.loc[ind,'IncidentTypeDescription1']).add_to(torn_inc_map)
torn_inc_map